<a href="https://colab.research.google.com/github/leonardoLavagna/Kaggle-Competition/blob/main/TrainingCampV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [25]:
# Standard imports
import time
import os
import copy
import tqdm.notebook as tq
from csv import writer

import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

In [4]:
# Configuration
IMAGE_SIZE = 224
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
PATH = '/content'
BATCHES = 16
NUM_WORKERS = 2
EPOCHS = 25
RANDOM_STATE = 1234
N_CLASSES = 7
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# to import the images on Google Colab execute the following command
#!git clone https://github.com/leonardoLavagna/Kaggle-Competition.git

Cloning into 'Kaggle-Competition'...
remote: Enumerating objects: 5227, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 5227 (delta 2), reused 4 (delta 0), pack-reused 5218
Receiving objects: 100% (5227/5227), 129.66 MiB | 43.52 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# dictionary of classes
diz ={"battleships":0,"coast-guard":1,"containerships":2,"cruise-ships":3,"drilling-rigs":4,"motor-yachts":5,"submarines":6}

# Load the data

In [5]:
df = pd.read_csv(PATH+"/train.csv")
df["path"] = "/content/train/" + df["category_name"] + "/" + df["file_name"]

In [6]:
# check
df.head()

,file_name,category_name,category_id,path
0,2c24dabe.jpg,motor-yachts,5,/content/train/motor-yachts/2c24dabe.jpg
1,7dcf4553.jpg,submarines,6,/content/train/submarines/7dcf4553.jpg
2,33e34b0c.jpg,motor-yachts,5,/content/train/motor-yachts/33e34b0c.jpg
3,3ac0a11a.jpg,motor-yachts,5,/content/train/motor-yachts/3ac0a11a.jpg
4,f6e59377.jpg,coast-guard,1,/content/train/coast-guard/f6e59377.jpg


In [7]:
df_train, df_val = train_test_split(df, stratify=df.category_id, test_size=0.2, random_state=RANDOM_STATE)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)

In [ ]:
import torchvision.transforms as T

batch_size = 16
num_workers = 2 # don't touch

In [8]:
input_size = (IMAGE_SIZE, IMAGE_SIZE)

In [9]:
class ShipDataset:
    def __init__(self, root, df, transform):
        self.root = root
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        cat = self.df.category_name[index]
        cat_id = self.df.category_id[index]
        file_name = self.df.file_name[index]
        img_path = os.path.join(self.root, cat, file_name)
  
        with open(img_path, "rb") as fp:
          img = Image.open(fp).convert("RGB")

        img = self.transform(img)
        return img, cat_id

In [10]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(input_size),
                            transforms.ToTensor(),
                            transforms.Normalize(MEAN, STD)
                            ])
# Define the ShipDataset, it will return a tuple: (image, label)
train_dataset = ShipDataset(PATH+"/images/train/", df_train, transform=train_transform)
print("len train_dataset", len(train_dataset))

len train_dataset 1847


In [12]:
val_transform = transforms.Compose([transforms.Resize(input_size),
                            transforms.ToTensor(),
                            transforms.Normalize(MEAN, STD)
                            ])

val_dataset = ShipDataset(PATH+"/images/train", df_val, transform=val_transform)
print("len val_dataset", len(val_dataset))

len val_dataset 462


In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=BATCHES,
                            num_workers=NUM_WORKERS,
                            shuffle=True,   
                            drop_last=True   
                            )

val_loader = torch.utils.data.DataLoader(val_dataset,
                            batch_size=BATCHES,
                            num_workers=NUM_WORKERS,
                            shuffle=False   
                            )

In [15]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [23]:
def train(model,cost, optimizer_ft,epochs):
    # send model to the GPU!
    n = len(train_loader)
    for epoch in range(epochs):
        # Train mode
        model.train()
        for iter, (image_batch, label_batch) in enumerate(tq.tqdm(train_loader)):
            # Stop to accumulate gradients
            optimizer_ft.zero_grad()

            image_batch = image_batch.to(DEVICE)
            label_batch = label_batch.to(DEVICE)
            # compute the logits and squeeze the last dimension
            logits = model(image_batch).squeeze()
            # Compute the mini-batch cost
            cost_tensor = cost(logits, label_batch)
            # sometimes print the loss
            if iter % int(0.25*n) == 0:
                print(f"Epoch {epoch}, iter {iter}/{n} Train cost {cost_tensor.item():.6f}")
            # Gradient computation!
            cost_tensor.backward()
            # Optimizer step!
            optimizer_ft.step()
            
        # Eval the model after every epoch
        meanf1score, y_true, y_pred = eval(model, val_loader)
        print(f"Val meanF1score: {meanf1score*100:.4f}")


def eval(model, loader):
    y_pred = []
    y_true = []
    # eval mode
    model.eval()
    for images, labels in loader:
        images = images.to(DEVICE)
        with torch.no_grad():
            logits = model(images)
        y_pred += list(torch.argmax(logits,1).cpu().numpy())
        y_true += list(labels.cpu().numpy())

    meanf1score = f1_score(y_true, y_pred, average="macro")

    return meanf1score, y_true, y_pred

In [24]:
model = model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.999))
#learning_rate = 1e-4
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
cost = torch.nn.CrossEntropyLoss()
train(model,cost,optimizer,30)

  0%|          | 0/57 [00:00<?, ?it/s]

Epoch 0, iter 0/57 Train cost 0.742157
Epoch 0, iter 14/57 Train cost 1.615928
Epoch 0, iter 28/57 Train cost 0.928734
Epoch 0, iter 42/57 Train cost 1.022023
Epoch 0, iter 56/57 Train cost 0.988636
Val meanF1score: 54.5860


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch 1, iter 0/57 Train cost 0.974823
Epoch 1, iter 14/57 Train cost 0.706843
Epoch 1, iter 28/57 Train cost 0.710596
Epoch 1, iter 42/57 Train cost 0.524517
Epoch 1, iter 56/57 Train cost 0.805807
Val meanF1score: 55.3289


  0%|          | 0/57 [00:00<?, ?it/s]

...

Epoch 27, iter 0/57 Train cost 0.134844
Epoch 27, iter 14/57 Train cost 0.317766
Epoch 27, iter 28/57 Train cost 0.156788
Epoch 27, iter 42/57 Train cost 0.319485
Epoch 27, iter 56/57 Train cost 0.223123
Val meanF1score: 77.8476


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch 28, iter 0/57 Train cost 0.721866
Epoch 28, iter 14/57 Train cost 0.368572
Epoch 28, iter 28/57 Train cost 0.707175
Epoch 28, iter 42/57 Train cost 0.510767
Epoch 28, iter 56/57 Train cost 0.237726
Val meanF1score: 72.4549


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch 29, iter 0/57 Train cost 0.380575
Epoch 29, iter 14/57 Train cost 0.396330
Epoch 29, iter 28/57 Train cost 0.617053
Epoch 29, iter 42/57 Train cost 0.180779
Epoch 29, iter 56/57 Train cost 0.418931
Val meanF1score: 77.6892


# Testing

In [33]:
test_dataset = ImageFolder("/content/test", transform=val_transform)

In [35]:
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=BATCHES,
                                          num_workers=NUM_WORKERS,
                                          shuffle=False)

In [36]:
# check
test_dataset.samples[:3]

[('/content/test/test/000c110b.jpg', 0),
 ('/content/test/test/00268327.jpg', 0),
 ('/content/test/test/008fcbc9.jpg', 0)]

In [38]:
y_pred = []
# eval mode
model.eval()
for images, labels in test_loader:
    images = images.to(DEVICE)
    with torch.no_grad():
        logits = model(images)
    y_pred += list(torch.argmax(logits,1).cpu().numpy())

# Submission file

In [40]:
df_submission = pd.DataFrame()
df_submission["file_name"] = [os.path.basename(p[0]) for p in test_dataset.samples]
df_submission["category_id"] = y_pred
df_submission.to_csv("submission.csv", index=False)